In [41]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import timedelta
from matplotlib.ticker import PercentFormatter

Lendo as bases

In [42]:
crm = pd.read_excel("/content/drive/MyDrive/Acelera/Bases tratadas/base_crm_tratada.xlsx")

In [43]:
cadastro = pd.read_excel("/content/drive/MyDrive/Acelera/Bases tratadas/base_cadastro_conta_digital_tratada.xlsx")

In [44]:
google_ads = pd.read_excel("/content/drive/MyDrive/Acelera/Bases tratadas/base_google_tratada.xlsx")

In [45]:
meta_ads = pd.read_excel("/content/drive/MyDrive/Acelera/Bases tratadas/base_metaads_tratada.xlsx")

In [46]:
google_analytics = pd.read_excel("/content/drive/MyDrive/Acelera/Bases tratadas/base_google_analytics_tratada.xlsx")

Quantidade de novos clientes

In [47]:
total_novos_clientes = cadastro['mês da abertura'].count()
total_novos_clientes

512

In [48]:
novos_clientes = cadastro['mês da abertura'].value_counts().reset_index()
novos_clientes.columns = ["mês da abertura", "clientes_novos"]
novos_clientes

,mês da abertura,clientes_novos
0,2023-01,129
1,2023-02,119
2,2022-12,98
3,2022-11,71
4,2022-10,60
5,2022-09,35


## Receitas

In [49]:
crm[crm['Conversões']=='Sim']['Receita'].sum().round(2)

9735383.85

In [50]:
receita = pd.DataFrame(crm[(crm['Conversões']=='Sim')].groupby('Mês do lead')['Receita'].sum().reset_index())
receita

,Mês do lead,Receita
0,2022-09,1023481.35
1,2022-10,1271012.25
2,2022-11,3017722.05
3,2022-12,1159978.20
4,2023-01,1701675.00
5,2023-02,1561515.00


### Valor realizado em 2023

In [51]:
crm[(crm['Conversões']=='Sim') & (crm['Data do Lead'].dt.year ==2023)]['Receita'].sum()

3263190.0

In [52]:
realizado2023 = pd.DataFrame(crm[(crm['Conversões']=='Sim') & (crm['Data do Lead'].dt.year ==2023)].groupby('Mês do lead')['Receita'].sum().reset_index())
realizado2023

,Mês do lead,Receita
0,2023-01,1701675.0
1,2023-02,1561515.0


In [53]:
realizado = crm[(crm['Conversões']=='Sim') & (crm['Data do Lead'].dt.year ==2023)]['Receita'].sum()

In [54]:
30000000 - realizado

26736810.0

# Despesas

Custo de tráfego pago

In [55]:
google_ads['Custo'].sum()

289683.9

In [56]:
meta_ads['Custo'].sum()

1103130.2

In [57]:
custo_marketing = google_ads['Custo'].sum() + meta_ads['Custo'].sum()
custo_marketing

1392814.1

In [58]:
google_ads

,Dia,Tipo de Campanha,Produto/Anúncio,Impressões do anúncio,Cliques,Custo,Conversões,Receita,Mês
0,2022-09-01,Institucional,Empréstimo,1080,150,300.6,0,0,2022-09
1,2022-09-01,Remarketing,Empréstimo,19960,100,80.2,0,0,2022-09
2,2022-09-02,Remarketing,Empréstimo,19650,110,87.9,0,0,2022-09
3,2022-09-02,Institucional,Empréstimo,890,110,161.6,0,0,2022-09
4,2022-09-03,Remarketing,Empréstimo,20510,160,103.2,0,0,2022-09
...,...,...,...,...,...,...,...,...,...
603,2023-02-27,Conversão-Youtube,Empréstimo,22230,160,720.0,0,0,2023-02
604,2023-02-28,Remarketing,Empréstimo,45690,790,378.0,0,0,2023-02
605,2023-02-28,Institucional,Empréstimo,2070,450,380.2,0,0,2023-02
606,2023-02-28,Conversão-Youtube,Empréstimo,18960,160,716.0,0,0,2023-02


In [60]:
custo_google = google_ads.groupby(google_ads['Mês'])['Custo'].sum().reset_index()
custo_google.columns = ["Mês do lead", "custo google ads"]
custo_meta = meta_ads.groupby(meta_ads['mês do lead'])['Custo'].sum().reset_index()
custo_meta.columns = ["Mês do lead", "custo meta ads"]

In [61]:
custo_anuncios = custo_google.merge(custo_meta, how='left')
custo_anuncios['custo total'] = custo_anuncios["custo google ads"] + custo_anuncios["custo meta ads"]
custo_anuncios.round(2)
custo_anuncios['Mês do lead'] = custo_anuncios['Mês do lead'].astype('object')

In [62]:
custo_anuncios

,Mês do lead,custo google ads,custo meta ads,custo total
0,2022-09,19747.9,106171.0,125918.9
1,2022-10,37308.7,192543.5,229852.2
2,2022-11,42643.8,264467.3,307111.1
3,2022-12,54064.2,189053.9,243118.1
4,2023-01,63057.9,172831.8,235889.7
5,2023-02,72861.4,178062.7,250924.1


Salário vendedores

In [63]:
salario_vendedores = crm[crm['Conversões']=='Sim'].groupby(['Mês do lead'])['Receita'].sum().reset_index()
salario_vendedores['salário fixo'] = 1500*5
salario_vendedores['salário variável'] = (salario_vendedores['Receita']*0.005*5).round(2)
salario_vendedores['salário total'] = salario_vendedores['salário fixo'] + salario_vendedores['salário variável']
salario_vendedores

,Mês do lead,Receita,salário fixo,salário variável,salário total
0,2022-09,1023481.35,7500,25587.03,33087.03
1,2022-10,1271012.25,7500,31775.31,39275.31
2,2022-11,3017722.05,7500,75443.05,82943.05
3,2022-12,1159978.20,7500,28999.46,36499.46
4,2023-01,1701675.00,7500,42541.88,50041.88
5,2023-02,1561515.00,7500,39037.88,46537.88


In [64]:
total_salarios_vendedores = salario_vendedores['salário total'].sum()
total_salarios_vendedores

288384.61

In [65]:
(salario_vendedores['salário total'].sum()/5/6).round(2)

9612.82

###CAC = (investimento em marketing + investimento em vendas)/número de novos clientes

In [66]:
((custo_marketing + total_salarios_vendedores)/total_novos_clientes).round(2)

3283.59

In [68]:
custo_anuncios[['Mês do lead', 'custo total']].merge(salario_vendedores[['Mês do lead', 'salário total']].merge(), how = 'left')

,Mês do lead,custo total,salário total
0,2022-09,125918.9,33087.03
1,2022-10,229852.2,39275.31
2,2022-11,307111.1,82943.05
3,2022-12,243118.1,36499.46
4,2023-01,235889.7,50041.88
5,2023-02,250924.1,46537.88


In [ ]:
custo_anuncios.info()

In [ ]:
salario_vendedores.info()